# Classification

In [1]:
import nibabel
import numpy as np
from nilearn.input_data import NiftiMasker
from nilearn import image
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression

/Users/leonglab/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


### Set-up classification

In [2]:
subjIDs = ["2013","2015","2016","2017","2018","2019","2020","2021","2022","2024", 
"2027","2028","2029","2030","2031","2032","2033","2034","2035","2036","2038",
"2039","2040","2041","2042","2043","2044","2045","2046","2047"]

mask_file = "vvs";
design = "cue_align_native_minus1"
classifier_name = "log_l1" # Or 'SVC', 'log_l1', log_l1_50 or log_l2

dir_input = "../../3_results/2_glm/" + design
dir_label = "z_trial_labels"                   # needs to be in trial space
 
dir_traininput ="../../3_results/2_glm/align_native_delete1"
dir_trainlabel ="z_trial_labels_TR"               # needs to be in TR space 

### Run Classification

In [ ]:
for subjID in subjIDs:
    print("Running Subj: %s" % subjID)
    
    #############################################################################
    #                     Always train on the same lag                         #
    #############################################################################
        
    # fMRI path
    run5_path = dir_traininput + "/" + "sub-" + subjID + "/run5.feat/filtered_func_data.nii.gz"
    run6_path = dir_traininput + "/" + "sub-" + subjID + "/run6.feat/filtered_func_data.nii.gz"
    
    # mask path
    mask_path = "../../2_fmri/masks/align_native/" + mask_file + "/" + "sub-" + subjID + "_run1.nii.gz"
    
    # Labels
    label_run5_path = dir_trainlabel + "/" + subjID + "/Add0/run5_label.txt"
    label_run6_path = dir_trainlabel + "/" + subjID + "/Add0/run6_label.txt"
    
    # Load labels and make target
    run5_labels = np.recfromcsv(label_run5_path, delimiter=" ")
    run5_target = run5_labels['labels']
    run5_condition_mask = np.logical_or(run5_labels['labels'] == 1, run5_labels['labels'] == 2)
    run5_target = run5_target[run5_condition_mask]
    
    run6_labels = np.recfromcsv(label_run6_path, delimiter=" ")
    run6_target = run6_labels['labels']
    run6_condition_mask = np.logical_or(run6_labels['labels'] == 1, run6_labels['labels'] == 2)
    run6_target = run6_target[run6_condition_mask]

    # Load MRI data
    nifti_masker = NiftiMasker(mask_img=mask_path,standardize=True)
    run5_fmri_masked = nifti_masker.fit_transform(run5_path)    
    run6_fmri_masked = nifti_masker.fit_transform(run6_path)
    
    # Need to make sure labels is same size as fMRI data
    toPad = run5_fmri_masked.shape[0] - run5_condition_mask.size
    padding = np.zeros(toPad, dtype = bool)
    run5_condition_mask = np.append(run5_condition_mask,padding)

    toPad = run6_fmri_masked.shape[0] - run6_condition_mask.size
    padding = np.zeros(toPad, dtype = bool)
    run6_condition_mask = np.append(run6_condition_mask,padding)
    
    # subset data
    run5_fmri_masked = run5_fmri_masked[run5_condition_mask]
    run6_fmri_masked = run6_fmri_masked[run6_condition_mask]
        
    # Concatenate Runs
    combined_runs = np.vstack((run5_fmri_masked,run6_fmri_masked))
        
    # Concatenate Labels
    combined_labels = np.hstack((run5_target,run6_target))
    
    # Decoder
    if classifier_name == "log_l1":
        print("Training using log_l1")
        classifier = LogisticRegression(C=1., penalty="l1", solver='liblinear',tol=0.00001)  
        
    # Train
    classifier.fit(combined_runs, combined_labels)
        
    # Create toPrint 
    AllShifts = np.zeros((0,5));
                    
    # Initialize variables
    task_path = []
    task_label_path = []
    task_label = []
    task_target = []
    task_condition_mask = []
    task_target = []
    task_trial = []
    task_fmri_masked = []
    combined_task = []
    combined_task_trial = []

    # Load run 1 to run 4 
    for r in range(0,4):                
        task_path.append(dir_input + "/" + "sub-" + subjID + "/run" + str(r+1) + "_copemap_scaled.nii.gz")        
        task_label_path.append(dir_label + "/" + subjID  + "/run" + str(r+1) + "_label.txt")
        task_label.append(np.recfromcsv(task_label_path[r], delimiter=" "))
        task_target.append(task_label[r]['condition'])
        task_condition_mask.append(np.logical_or(task_label[r]['condition'] == 1, task_label[r]['condition'] == 2))
        task_target[r] = task_target[r][task_condition_mask[r]]
        task_trial.append(task_label[r][task_condition_mask[r]])

        # Stopped here because of mask issues
        task_fmri_masked.append(nifti_masker.fit_transform(task_path[r]))
        task_fmri_masked[r] = task_fmri_masked[r][task_condition_mask[r]]

    #Concatenate Runs and Labels 
    combined_task = np.vstack([task_fmri_masked[0],task_fmri_masked[1],task_fmri_masked[2],task_fmri_masked[3]])
    combined_task_trial = np.vstack([task_trial[0].tolist(),task_trial[1].tolist(),task_trial[2].tolist(),task_trial[3].tolist()])

    # Classify
    prob = classifier.predict_proba(combined_task)

    # Output to CSV
    toPrint = np.column_stack([combined_task_trial,prob[:,1]])
    AllShifts = np.vstack([AllShifts,toPrint]);

    if not os.path.exists("../../3_results/6_TrialClassification/" + design + "_copemap_scaled/TrainTrial/" + mask_file + "/" + classifier_name):
        os.makedirs("../../3_results/6_TrialClassification/" + design + "_copemap_scaled/TrainTrial/" + mask_file + "/" + classifier_name)

    outputfile = "../../3_results/6_TrialClassification/" + design + "_copemap_scaled/TrainTrial/" + mask_file + "/" + classifier_name + "/" + subjID + ".csv"
    np.savetxt(outputfile, AllShifts, delimiter=",")

Running Subj: 2013
Training using log_l1
Running Subj: 2015
Training using log_l1
Running Subj: 2016
Training using log_l1
Running Subj: 2017
Training using log_l1
Running Subj: 2018
Training using log_l1
Running Subj: 2019
Training using log_l1
Running Subj: 2020
Training using log_l1
Running Subj: 2021
Training using log_l1
Running Subj: 2022
Training using log_l1
Running Subj: 2024
Training using log_l1
Running Subj: 2027
Training using log_l1
Running Subj: 2028
Training using log_l1
Running Subj: 2029
Training using log_l1
Running Subj: 2030
Training using log_l1
Running Subj: 2031
Training using log_l1
Running Subj: 2032
Training using log_l1
Running Subj: 2033
Training using log_l1
Running Subj: 2034
Training using log_l1
Running Subj: 2035
Training using log_l1
Running Subj: 2036
Training using log_l1
Running Subj: 2038
